In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [8]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train.shape,x_test.shape

((60000, 28, 28), (10000, 28, 28))

In [9]:
x_train =x_train.reshape(-1,28*28).astype("float32") / 255.0 # -1的意思是保留60000这个维度,/255是为了让它保持在0到1之间
x_test = x_test.reshape(-1,28*28).astype("float32") / 255.0 

上面的x_train 和x_test还是numpy数组，现在我们来把他们转化成tensor张量

In [10]:
x_train = tf.convert_to_tensor(x_train)
x_test = tf.convert_to_tensor(x_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

其实上面这个操作步骤并不需要我们手动来做，tensorflow内部会自动转换

sequential API

Sequential api 只能单个输入，单个输出，不是很灵活

In [30]:
model = keras.Sequential([layers.Dense(512,activation='relu'),
                         layers.Dense(256,activation='relu',name='second_layer'),
                         layers.Dense(10)])

In [33]:
model = keras.Sequential([keras.Input(shape=28*28),#加了这个就可以打印model.summary() 
                         layers.Dense(512,activation='relu'),
                         layers.Dense(256,activation='relu',name='second_layer'),
                         layers.Dense(10)])

In [34]:
model = keras.Sequential()
model.add(keras.Input(shape=784))
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(256,activation='relu',name='second_layer'))
model.add(layers.Dense(10))

In [12]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             optimizer=keras.optimizers.Adam(learning_rate=0.001),
             metrics=['accuracy'])

In [14]:
model.fit(x_train,y_train,batch_size=32,epochs=30,verbose=2)

Epoch 1/30
1875/1875 - 5s - loss: 0.1872 - accuracy: 0.9429 - 5s/epoch - 3ms/step
Epoch 2/30
1875/1875 - 4s - loss: 0.0778 - accuracy: 0.9755 - 4s/epoch - 2ms/step
Epoch 3/30
1875/1875 - 5s - loss: 0.0575 - accuracy: 0.9815 - 5s/epoch - 2ms/step
Epoch 4/30
1875/1875 - 4s - loss: 0.0408 - accuracy: 0.9870 - 4s/epoch - 2ms/step
Epoch 5/30
1875/1875 - 5s - loss: 0.0334 - accuracy: 0.9894 - 5s/epoch - 2ms/step
Epoch 6/30
1875/1875 - 5s - loss: 0.0267 - accuracy: 0.9916 - 5s/epoch - 2ms/step
Epoch 7/30
1875/1875 - 4s - loss: 0.0247 - accuracy: 0.9923 - 4s/epoch - 2ms/step
Epoch 8/30
1875/1875 - 4s - loss: 0.0208 - accuracy: 0.9938 - 4s/epoch - 2ms/step
Epoch 9/30
1875/1875 - 4s - loss: 0.0190 - accuracy: 0.9939 - 4s/epoch - 2ms/step
Epoch 10/30
1875/1875 - 4s - loss: 0.0171 - accuracy: 0.9948 - 4s/epoch - 2ms/step
Epoch 11/30
1875/1875 - 4s - loss: 0.0163 - accuracy: 0.9949 - 4s/epoch - 2ms/step
Epoch 12/30
1875/1875 - 4s - loss: 0.0158 - accuracy: 0.9950 - 4s/epoch - 2ms/step
Epoch 13/30
1

In [16]:
model.evaluate(x_test,y_test,batch_size=32,verbose=2)

313/313 - 0s - loss: 0.2068 - accuracy: 0.9802 - 358ms/epoch - 1ms/step


[0.20678061246871948, 0.9801999926567078]

如何得到特定层的输出细节

In [28]:
model = keras.Model(inputs=model.inputs,
                    outputs=model.layers[-2].output)
feature = model.predict(x_train)
feature.shape

(60000, 256)

In [35]:
model = keras.Model(inputs=model.inputs,
                    outputs=model.get_layer('second_layer').output)
feature = model.predict(x_train)
feature.shape

(60000, 256)

Functional API

functinal api比较灵活，可以支持多种输入，多种输出

In [23]:
inputs = keras.Input(shape=(784))
x = layers.Dense(512,activation='relu',name='first_layer')(inputs)# name 可加可不加
x= layers.Dense(256,activation='relu')(x)
outputs= layers.Dense(10,activation='softmax')(x)
model = keras.Model(inputs=inputs,outputs=outputs)

In [26]:
model.summmary()

AttributeError: 'Functional' object has no attribute 'summmary'

In [24]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
             optimizer=keras.optimizers.Adam(learning_rate=0.001),
             metrics=['accuracy'])

In [25]:
model.fit(x_train,y_train,batch_size=32,epochs=5,verbose=2)

Epoch 1/5
1875/1875 - 5s - loss: 0.1865 - accuracy: 0.9435 - 5s/epoch - 2ms/step
Epoch 2/5
1875/1875 - 4s - loss: 0.0790 - accuracy: 0.9751 - 4s/epoch - 2ms/step
Epoch 3/5
1875/1875 - 4s - loss: 0.0544 - accuracy: 0.9824 - 4s/epoch - 2ms/step
Epoch 4/5
1875/1875 - 5s - loss: 0.0416 - accuracy: 0.9866 - 5s/epoch - 2ms/step
Epoch 5/5
1875/1875 - 5s - loss: 0.0336 - accuracy: 0.9896 - 5s/epoch - 2ms/step
